In [1]:
import pandas as pd

df_train= pd.read_csv("/kaggle/input/titanic20250616/train.csv")
df_test= pd.read_csv("/kaggle/input/titanic20250616/test.csv")

df_test_PassengerId=df_test["PassengerId"]
df_train=df_train.drop(['PassengerId',"Name","Ticket","Cabin"],axis=1)
df_test=df_test.drop(['PassengerId',"Name","Ticket","Cabin"],axis=1)

most_frequent = df_train['Embarked'].mode()[0]
print(most_frequent)
df_train['Embarked']=df_train['Embarked'].fillna(most_frequent)
df_train['Age']=df_train['Age'].fillna(df_train['Age'].mean())
df_test['Age']=df_test['Age'].fillna(df_test['Age'].mean())
df_test['Fare']=df_test['Fare'].fillna(df_test['Fare'].mean())

print(df_train.isnull().sum())
print(df_test.isnull().sum())
print(df_train.dtypes)
print(df_test.dtypes)

df_train = pd.get_dummies(df_train)
df_test = pd.get_dummies(df_test)

print(df_train.isnull().sum())
print(df_test.isnull().sum())

print(df_train.dtypes)
print(df_test.dtypes)

S
Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64
Survived      int64
Pclass        int64
Sex          object
Age         float64
SibSp         int64
Parch         int64
Fare        float64
Embarked     object
dtype: object
Pclass        int64
Sex          object
Age         float64
SibSp         int64
Parch         int64
Fare        float64
Embarked     object
dtype: object
Survived      0
Pclass        0
Age           0
SibSp         0
Parch         0
Fare          0
Sex_female    0
Sex_male      0
Embarked_C    0
Embarked_Q    0
Embarked_S    0
dtype: int64
Pclass        0
Age           0
SibSp         0
Parch         0
Fare          0
Sex_female    0
Sex_male      0
Embarked_C    0
Embarked_Q    0
Embarked_S    0
dtype: int64
Survived        int64
Pclass          int64
Age           float64
SibSp 

In [2]:
df_train['Age']=df_train['Age'].fillna(df_train['Age'].mean())
df_test['Age']=df_test['Age'].fillna(df_test['Age'].mean())
df_test['Fare']=df_test['Fare'].fillna(df_test['Fare'].mean())
print(df_train.isnull().sum())
print(len(df_train))
print(df_test.isnull().sum())
print(len(df_test))

Survived      0
Pclass        0
Age           0
SibSp         0
Parch         0
Fare          0
Sex_female    0
Sex_male      0
Embarked_C    0
Embarked_Q    0
Embarked_S    0
dtype: int64
891
Pclass        0
Age           0
SibSp         0
Parch         0
Fare          0
Sex_female    0
Sex_male      0
Embarked_C    0
Embarked_Q    0
Embarked_S    0
dtype: int64
418


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [4]:
X = df_train.drop('Survived', axis=1)
y = df_train['Survived']

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [6]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'max_features': ['sqrt', 'log2']
}

grid_search = GridSearchCV(
    estimator=RandomForestClassifier(random_state=42),
    param_grid=param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1
)

grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_
print("Best Parameters:", grid_search.best_params_)


Best Parameters: {'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 300}


In [7]:
model = RandomForestClassifier(
    n_estimators=300,
    max_depth=10,
    max_features='sqrt',
    min_samples_split=2,
    min_samples_leaf=2,
    class_weight='balanced',
    random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(class_weight='balanced', max_depth=10,
                       min_samples_leaf=2, n_estimators=300, random_state=42)

In [8]:
y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.8156424581005587
Confusion Matrix:
 [[98 12]
 [21 48]]
Classification Report:
               precision    recall  f1-score   support

           0       0.82      0.89      0.86       110
           1       0.80      0.70      0.74        69

    accuracy                           0.82       179
   macro avg       0.81      0.79      0.80       179
weighted avg       0.81      0.82      0.81       179



In [9]:
y_test_pred = model.predict(df_test)

submission = pd.DataFrame({
    'PassengerId': df_test_PassengerId,
    'Survived': y_test_pred
})

submission.to_csv('submission3.csv', index=False)
